In [1]:
!pip uninstall -y scikit-learn scikeras
!pip install scikit-learn==1.4.2 scikeras==0.13.0

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 89.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from scikeras.wrappers import KerasRegressor

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

2025-05-06 15:23:22.909973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746545003.168546      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746545003.242095      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
training_data = pd.read_csv('/kaggle/input/mekong-training-testing-set/merged_train.csv')
testing_data = pd.read_csv('/kaggle/input/mekong-training-testing-set/merged_test.csv')

In [5]:
# Change the type into date time
def into_datetime(df):
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year

In [6]:
into_datetime(training_data)
into_datetime(testing_data)

In [7]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 37 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   AWEInsh    223 non-null    float64       
 1   AWEIsh     223 non-null    float64       
 2   MNDWI      223 non-null    float64       
 3   NDWI       223 non-null    float64       
 4   SR_B1      223 non-null    float64       
 5   SR_B2      223 non-null    float64       
 6   SR_B3      223 non-null    float64       
 7   SR_B4      223 non-null    float64       
 8   SR_B5      223 non-null    float64       
 9   SR_B7      223 non-null    float64       
 10  WI1        223 non-null    float64       
 11  WI2        223 non-null    float64       
 12  date       223 non-null    datetime64[ns]
 13  pivot      223 non-null    int64         
 14  reducer    223 non-null    object        
 15  source     223 non-null    object        
 16  Station    223 non-null    object        
 1

In [8]:
testing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 37 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   AWEInsh    35 non-null     float64       
 1   AWEIsh     35 non-null     float64       
 2   MNDWI      35 non-null     float64       
 3   NDWI       35 non-null     float64       
 4   SR_B1      35 non-null     float64       
 5   SR_B2      35 non-null     float64       
 6   SR_B3      35 non-null     float64       
 7   SR_B4      35 non-null     float64       
 8   SR_B5      35 non-null     float64       
 9   SR_B7      35 non-null     float64       
 10  WI1        35 non-null     float64       
 11  WI2        35 non-null     float64       
 12  date       35 non-null     datetime64[ns]
 13  pivot      35 non-null     int64         
 14  reducer    35 non-null     object        
 15  source     35 non-null     object        
 16  Station    35 non-null     object        
 17 

In [9]:
features = [ 'SR_B5', 'SR_B7',  'Latitude', 'Longitude', 
            'AWEInsh', 'AWEIsh', 'MNDWI', 'NDWI', 'SR_B1', 'SR_B2', 'SR_B3', 
            'SR_B4', 'WI1', 'WI2', 'month', 'year']

In [10]:
target = 'max'

In [11]:
# Split data
X_train = training_data[features]
y_train = training_data[target]
X_test = testing_data[features]
y_test = testing_data[target]

In [12]:
# Define model
model = RandomForestRegressor(random_state=42)

In [13]:
# Apply RFE
rfe = RFE(estimator=model, n_features_to_select=15)  # Choose number of features
X_train_rfe = rfe.fit_transform(X_train, y_train)
X_test_rfe = rfe.transform(X_test)

In [14]:
# 3. Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_rfe)
X_test_scaled = scaler.transform(X_test_rfe)

In [15]:
def build_model(hidden_layers=2, neurons=128, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(Input(shape=(X_train_rfe.shape[1],)))
    
    for _ in range(hidden_layers):
        model.add(Dense(neurons, kernel_regularizer=l2(0.001)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))
    
    model.add(Dense(1))  # Output layer
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

In [16]:
# ========== STEP 3: Set up EarlyStopping ==========
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


# ========== STEP 4: Wrap with KerasRegressor ==========
mlp = KerasRegressor(
    model=build_model,
    verbose=0,
    callbacks=[early_stop],
    validation_split=0.2
)

In [17]:
param_grid = {
    'mlp__model__hidden_layers': [2, 3, 4],
    'mlp__model__neurons': [64, 128, 256],
    'mlp__model__dropout_rate': [0.1, 0.2],
    'mlp__model__learning_rate': [0.001, 0.0005],
    'mlp__batch_size': [16, 32],
    'mlp__epochs': [150, 200]
}

In [18]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', mlp)
])

In [19]:
# Cross-validation strategy
cv = KFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=cv,
    scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False),
    verbose=2,
    n_jobs=-1
)
# Fit grid search
grid_result = grid.fit(X_train_rfe, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


2025-05-06 15:23:41.376402: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-06 15:23:41.409301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746545021.425227      56 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746545021.440781      56 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 15:23:41.444758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:17465

[CV] END mlp__batch_size=16, mlp__epochs=150, mlp__model__dropout_rate=0.1, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.001, mlp__model__neurons=64; total time=  14.5s
[CV] END mlp__batch_size=16, mlp__epochs=150, mlp__model__dropout_rate=0.1, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.001, mlp__model__neurons=128; total time=  11.3s
[CV] END mlp__batch_size=16, mlp__epochs=150, mlp__model__dropout_rate=0.1, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.001, mlp__model__neurons=128; total time=  12.9s
[CV] END mlp__batch_size=16, mlp__epochs=150, mlp__model__dropout_rate=0.1, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.001, mlp__model__neurons=256; total time=  12.7s
[CV] END mlp__batch_size=16, mlp__epochs=150, mlp__model__dropout_rate=0.1, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.0005, mlp__model__neurons=64; total time=  13.8s
[CV] END mlp__batch_size=16, mlp__epochs=150, mlp__model__dropout_rate=0.1, mlp__model__hid

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
2025-05-06 15:28:26.643419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746545306.692619   66731 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746545306.706446   66731 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squa

[CV] END mlp__batch_size=16, mlp__epochs=150, mlp__model__dropout_rate=0.1, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.001, mlp__model__neurons=64; total time=  13.0s
[CV] END mlp__batch_size=16, mlp__epochs=150, mlp__model__dropout_rate=0.1, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.001, mlp__model__neurons=64; total time=  13.8s
[CV] END mlp__batch_size=16, mlp__epochs=150, mlp__model__dropout_rate=0.1, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.001, mlp__model__neurons=128; total time=  12.8s
[CV] END mlp__batch_size=16, mlp__epochs=150, mlp__model__dropout_rate=0.1, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.001, mlp__model__neurons=256; total time=  13.2s
[CV] END mlp__batch_size=16, mlp__epochs=150, mlp__model__dropout_rate=0.1, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.0005, mlp__model__neurons=64; total time=  20.5s
[CV] END mlp__batch_size=16, mlp__epochs=150, mlp__model__dropout_rate=0.1, mlp__model__hidd

...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2025-05-06 16:21:10.067556: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746548470.117000  827585 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746548470.129821  827585 cuda_blas.cc:1418] Unable to register cuBLAS factory: Att

[CV] END mlp__batch_size=32, mlp__epochs=200, mlp__model__dropout_rate=0.2, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.001, mlp__model__neurons=256; total time=  14.3s
[CV] END mlp__batch_size=32, mlp__epochs=200, mlp__model__dropout_rate=0.2, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.0005, mlp__model__neurons=64; total time=  29.9s
[CV] END mlp__batch_size=32, mlp__epochs=200, mlp__model__dropout_rate=0.2, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.0005, mlp__model__neurons=128; total time=  24.7s
[CV] END mlp__batch_size=32, mlp__epochs=200, mlp__model__dropout_rate=0.2, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.0005, mlp__model__neurons=256; total time=  15.4s
[CV] END mlp__batch_size=32, mlp__epochs=200, mlp__model__dropout_rate=0.2, mlp__model__hidden_layers=3, mlp__model__learning_rate=0.001, mlp__model__neurons=64; total time=  20.7s
[CV] END mlp__batch_size=32, mlp__epochs=200, mlp__model__dropout_rate=0.2, mlp__model__h

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2025-05-06 16:21:55.114227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746548515.163333  837051 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746548515.178187  837051 cuda_blas.cc:1418] Unable to register cuBLAS factory: Att

[CV] END mlp__batch_size=32, mlp__epochs=200, mlp__model__dropout_rate=0.2, mlp__model__hidden_layers=2, mlp__model__learning_rate=0.0005, mlp__model__neurons=256; total time=  17.3s
[CV] END mlp__batch_size=32, mlp__epochs=200, mlp__model__dropout_rate=0.2, mlp__model__hidden_layers=3, mlp__model__learning_rate=0.001, mlp__model__neurons=64; total time=  18.7s
[CV] END mlp__batch_size=32, mlp__epochs=200, mlp__model__dropout_rate=0.2, mlp__model__hidden_layers=3, mlp__model__learning_rate=0.001, mlp__model__neurons=128; total time=  21.1s
[CV] END mlp__batch_size=32, mlp__epochs=200, mlp__model__dropout_rate=0.2, mlp__model__hidden_layers=3, mlp__model__learning_rate=0.001, mlp__model__neurons=256; total time=  15.7s
[CV] END mlp__batch_size=32, mlp__epochs=200, mlp__model__dropout_rate=0.2, mlp__model__hidden_layers=3, mlp__model__learning_rate=0.0005, mlp__model__neurons=64; total time=  10.7s
[CV] END mlp__batch_size=32, mlp__epochs=200, mlp__model__dropout_rate=0.2, mlp__model__hi

2025-05-06 16:23:09.841748: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746548589.869811  855517 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746548589.880657  855517 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2025-05-06 16:23:15.786517: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (

In [20]:
# Step 8: Evaluate
print("✅ Best Parameters:", grid_result.best_params_)

✅ Best Parameters: {'mlp__batch_size': 16, 'mlp__epochs': 200, 'mlp__model__dropout_rate': 0.2, 'mlp__model__hidden_layers': 2, 'mlp__model__learning_rate': 0.0005, 'mlp__model__neurons': 64}


In [21]:
best_model = grid_result.best_estimator_

In [22]:
y_train_pred = best_model.predict(X_train_rfe)
y_test_pred = best_model.predict(X_test_rfe)

In [23]:
def wmape(y_true, y_pred):
    return 100 * np.sum(np.abs(y_true - y_pred)) / (np.sum(np.abs(y_true)) + 1e-8)

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def evaluate(y_true, y_pred, label):
    print(f"\n📊 MLP Evaluation on {label}:")
    print(f"• RMSE  : {mean_squared_error(y_true, y_pred, squared=False):.4f}")
    print(f"• R²    : {r2_score(y_true, y_pred):.4f}")
    print(f"• MAE   : {mean_absolute_error(y_true, y_pred):.4f}")
    print(f"• WMAPE : {wmape(y_true, y_pred):.2f}%")

In [24]:
evaluate(y_train, y_train_pred, "Training Set")


📊 MLP Evaluation on Training Set:
• RMSE  : 5.9768
• R²    : 0.4865
• MAE   : 3.8591
• WMAPE : 36.13%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [25]:
evaluate(y_test, y_test_pred, "Testing Set")


📊 MLP Evaluation on Testing Set:
• RMSE  : 6.4384
• R²    : 0.2508
• MAE   : 4.3253
• WMAPE : 58.97%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
